In [ ]:
import pandas as pd
import random
import os
from tqdm import tqdm

In [ ]:
# load csv from 
# https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information/data

In [ ]:
# can take 10min+ to load
data_df = pd.read_csv('song_lyrics.csv')

In [ ]:
english_df = data_df[data_df['language'] == 'en']

In [ ]:
def get_words(lyrics, max_words):
    count = 0
    for i, lyric in enumerate(lyrics):
        count += len(lyric.split())
        if count > max_words:
            return lyrics[:i], i
    raise ValueError("Not enough words in lyrics to reach max_words")

In [ ]:
texts = {}
random.seed(42)

for tag in tqdm(english_df['tag'].unique()):
    lyrics = english_df[english_df['tag'] == tag]['lyrics'].tolist()
    random.shuffle(lyrics)

    training_text, i = get_words(lyrics, 10**7)
    lyrics = lyrics[i:]
    validation_text, i = get_words(lyrics, 10**6)
    lyrics = lyrics[i:]
    test_text, i = get_words(lyrics, 5*(10**6))

    texts[tag] = {
        'training': training_text,
        'validation': validation_text,
        'test': test_text
    }

In [ ]:
for tag in texts.keys():
    training = "\n\n".join(texts[tag]['training'])
    validation = "\n\n".join(texts[tag]['validation'])
    test = "\n\n".join(texts[tag]['test'])

    print(f"Tag: {tag}")
    print(f"Training data length: {len(training.split())}")
    print(f"Validation data length: {len(validation.split())}")
    print(f"Test data length: {len(test.split())}")

    if os.path.exists(tag):
        os.system(f"rm -rf {tag}")
    os.makedirs(tag)
    with open(f"{tag}/data.train", "w") as f:
        f.write(training)
    with open(f"{tag}/data.dev", "w") as f:
        f.write(validation)
    with open(f"{tag}/data.test", "w") as f:
        f.write(test)
